### Ingestion and calling the Claude API
The best way to pass Claude charts and graphs is to take advantage of its vision capabilities.

In [ ]:
# # pip install -qUr requirements.txt
# !pip install pdf2image==1.17.0
# !pip install PyMuPDF==1.24.5

In [ ]:
import boto3
import json
import base64
import os
from datetime import datetime
from IPython.display import Image
from botocore.exceptions import ClientError
import time
import pandas as pd
from tqdm.notebook import tqdm
import os
import cv2
import numpy as np

session = boto3.Session()
region = session.region_name

#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [ ]:
def get_completion(messages):
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
    }
    response = bedrock_client.converse(**converse_api_params)
    # Extract the generated text content from the response
    return response['output']['message']['content'][0]['text']

In [ ]:

def formattedtime(seconds):
    #print(f"formattedtime({seconds})")
    final_time = time.strftime("%H:%M:%S", time.gmtime(float(seconds)))
    return f"{final_time}"

In [ ]:
question = "Test Question Here"

In [ ]:
# Now let's pass the first 20 of these images (in order) to Claude at once and ask it a question about the deck.
# Why 20? Currently, the Anthropic API only allows you to pass in a maximum of 20 images.
# While this number will likely increase over time, we have some helpful tips for how to manage it later in this recipe.


# question = "What was Twilio y/y revenue growth for fiscal year 2023?"

prompt = f"""You are a question answering assistant. Answer the given question if its evidence
is present in the input image in the <answer> tab else just output <answer> don't no </output> without adding
any additional text strictly.
<question>
{question}
</question>
<instructions>
- If answer is just not present just output <answer> don't know </output> and don't add any additional
text strictly in this case since you will be penalised for additional text .
- Strictly don't say things like 'I'm sorry, but I don't ' or
'I don't see any relevant information in the given images to answer this question.'
'I do not have enough information to answer this question'
etc. just say 'don't know'
</instructions>
"""

#question = "What was the non-GAAP gross margin?"



In [ ]:
image_dir = f"/data/split_video"
image_path_list = []
answer_total = ""
counter = 0
from tqdm.notebook import tqdm
for images in tqdm(os.listdir(image_dir)):
    # print(images)
    if images == ".ipynb_checkpoints":
        continue
    with open(f"{image_dir}/{images}", "rb") as f:
        img = f.read()

    content = [{"image": {"format": 'png', "source": {"bytes": img}}}]

    # Append the question to our images
    content.append({"text": prompt})

    messages = [
        {
            "role": 'user',
            "content": content
        },
        {
            "role": 'assistant',
            "content": [{"text":"<answer>:"}]
        }
    ]

    answer = get_completion(messages)
    time_stamp = int(images.split("_")[-1][:-4])


    start_time = formattedtime(time_stamp)

    end_time = formattedtime(time_stamp + 60)
    file = images[:images.find(".mp4")+4]


    answer_total = f"""{answer_total}\n\n image file - {file} \n start time - {start_time}

    end time - {end_time} \n \n

    Answer :- {answer}"""





In [ ]:
# file = "Lecture_20___Bayesian_Network-mod04lec20.mp4_49080.jpg"
# time_stamp = int(file.split("_")[-1][:-4])


# start_time = formattedtime(time_stamp)
# end_time = formattedtime(time_stamp + 60)

# start_time

In [ ]:
answer_total = answer_total.replace("don't know","")
answer_total = answer_total.replace("</answer>","")

In [ ]:
print(answer_total)

In [ ]:

prompt_final = f"""The given context has answer tags since the output is generated from previous conversations.
It may contain a lot of text saying answer is not present in the context and you need to ignore them.
Answer might be present in some of the tags. Use them to answer the question.
Here is the  context:
<context>
{answer_total}
</context>
Answer the question
<question>
{question}
</question>
Remember to enclose the answer in <answer> tags
Also generate start_time, end_time and image_file in start_time,end_time and image_file tags respectively which is the source of answer.
"""
messages = [
    {
        "role": 'user',
        "content": [
            {"text": prompt}

        ]
    }
]
answer = get_completion(messages)
print(answer)

In [ ]:
answer

---

# Evaluation

In [ ]:
import json
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# with open("../app/QA_PAIR.generated_output.jsonl", 'r') as my_file:
#     qa_pair = json.loads(my_file.read())

import json

data = []
with open("../app/generated_output.jsonl", 'r') as f:
    for line in f:
        data.append(json.loads(line))

# scores_list = []

In [ ]:
data[0].keys()

In [ ]:
claude_precision_list,claude_recall_list,claude_fmeasure_list  = [] , [], []
llama_precision_list,llama_recall_list,llama_fmeasure_list  = [] , [], []
multi_modal_precision_list,multi_modal_recall_list,multi_modal_fmeasure_list  = [] , [], []

for item in data:
    claude = item["generated_answer_claude3"]
    llama3 = item["llama3"]
    multi_model = item["multi_model"]
    ground_truth = item["ground_truth"]
    score_claude = scorer.score(ground_truth,claude)["rouge1"]
    score_llama3 = scorer.score(ground_truth,llama3)["rouge1"]
    score_multi_modal = scorer.score(ground_truth,multi_model)["rouge1"]
    # print(score_multi_modal.recall)
    claude_precision_list.append(score_claude.precision)
    claude_recall_list.append(score_claude.recall)
    claude_fmeasure_list.append(score_claude.fmeasure)
    llama_precision_list.append(score_llama3.precision)
    llama_recall_list.append(score_llama3.recall)
    llama_fmeasure_list.append(score_llama3.fmeasure)
    multi_modal_precision_list.append(score_multi_modal.precision)
    multi_modal_recall_list.append(score_multi_modal.recall)
    multi_modal_fmeasure_list.append(score_multi_modal.fmeasure)


    #     precision_list.append(item["rouge1"].precision)
#     recall_list.append(item["rouge1"].recall)
#     fmeasure_list.append(item["rouge1"].fmeasure)


In [ ]:
np.mean(claude_precision_list) , np.mean(claude_recall_list) , np.mean(claude_fmeasure_list)

In [ ]:
np.mean(llama_precision_list) , np.mean(llama_recall_list) , np.mean(llama_fmeasure_list)

In [ ]:
np.mean(multi_modal_precision_list) , np.mean(multi_modal_recall_list) , np.mean(multi_modal_fmeasure_list)

In [ ]:
multi_modal_recall_list

[]

In [ ]:
image_dir = f"/data/split_video"



time_frames = 40

for item in tqdm(qa_pair):
    question = item["question"]
    ground_truth = item["answer"]


    os.system(f"rm -r {image_dir}")
    os.system(f"mkdir -p {image_dir}")



    pathOut = image_dir
    input_video_path =f"/data/knowledgebase_video"
    count = 0
    counter = 1
    listing = os.listdir(input_video_path)
    vid =
    for vid in listing:
        full_vid_path = f"{input_video_path}/{vid}"
        vid = vid.replace(" ","_")
        cap = cv2.VideoCapture(full_vid_path)
        count = 0
        counter += 1
        success = True
        while success:
            success,image = cap.read()
            # print('read a new frame:',success)
            if success == False:
                break
            if count >= time_frames and count<=time_frames + 240:
                if count % 15 == 0:
                    #cv2.imwrite(pathOut + 'frame%d.jpg'%count,image)
                    #print(f"writing -- {pathOut}/{vid}_{count}.jpg")
                    cv2.imwrite(f"{pathOut}/{vid}_{count}.jpg",image)
            count+=1



    print("Done this !")


    image_path_list = []
    answer_total = ""
    counter = 0




    for images in tqdm(os.listdir(image_dir)):
        # print(images)
        if images == ".ipynb_checkpoints":
            continue
        with open(f"{image_dir}/{images}", "rb") as f:
            img = f.read()

        content = [{"image": {"format": 'png', "source": {"bytes": img}}}]

        # Append the question to our images
        content.append({"text": prompt})

        messages = [
            {
                "role": 'user',
                "content": content
            },
            {
                "role": 'assistant',
                "content": [{"text":"<answer>:"}]
            }
        ]

        answer = get_completion(messages)
        time_stamp = int(images.split("_")[-1][:-4])


        start_time = formattedtime(time_stamp)

        end_time = formattedtime(time_stamp + 60)
        file = images[:images.find(".mp4")+4]


        answer_total = f"""{answer_total}

        Answer :- {answer}"""


        answer_total = answer_total.replace("don't know","")
        answer_total = answer_total.replace("</answer>","")


        prompt_final = f"""The given context has answer tags since the output is generated from previous conversations.
        It may contain a lot of text saying answer is not present in the context and you need to ignore them.
        Answer might be present in some of the tags. Use them to answer the question.
        Here is the  context:
        <context>
        {answer_total}
        </context>
        Answer the question
        <question>
        {question}
        </question>
        Remember to enclose the answer in <answer> tags
        Also generate start_time, end_time and image_file in start_time,end_time and image_file tags respectively which is the source of answer.
        """

        messages = [
            {
                "role": 'user',
                "content": [
                    {"text": prompt_final}

                ]
            }
        ]
        answer = get_completion(messages)
        print(answer)